In [84]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from shutil import copyfile
from shutil import move
import re

data_dir = 'D:/Downloads/amazon/'

df = pd.read_csv(data_dir + 'train.csv')
df.head(100)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


['partly_cloudy', 'primary']

In [64]:
# copy images to the folder labeled with the right weather condition.  this is a preprocessing step for using Keras ImageDataGenerator.flow_from_directory.

def preprocessImageFileForIndex(index):
    source_filename = df.iloc[index]['image_name'] + '.jpg'
    #print(source_filename)
    #print(df.iloc[index]['tags'])

    data_src_dir = data_dir + 'train-jpg/'
    preprocess_train_dir = data_dir + 'preprocess/train-jpg/'

    for m in re.finditer('^[a-zA-Z_]+', df.iloc[index]['tags']):
        curr_tag = m.group(0)
        #print(curr_tag)
        if curr_tag == 'cloudy' or curr_tag == 'haze' or curr_tag == 'clear' or curr_tag == 'partly_cloudy':
            copyfile(data_src_dir + source_filename, preprocess_train_dir + curr_tag + '/'+ source_filename)
            break

In [65]:
# TODO it seems we only got about half of the total number of files. Why?
for index, row in df.iterrows():
    preprocessImageFileForIndex(index)


In [69]:
print('# File sizes')
for f in os.listdir(preprocess_train_dir):
    if not os.path.isdir(preprocess_train_dir + f):
        print(f.ljust(30) + str(round(os.path.getsize(preprocess_train_dir + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize(preprocess_train_dir+f+'/'+x)/1000000 for x in os.listdir(preprocess_train_dir + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))

# File sizes
clear                         0.0MB (17995 files)
cloudy                        0.0MB (2306 files)
haze                          0.0MB (1891 files)
partly_cloudy                 0.0MB (4480 files)


In [85]:
import random
# Move random samples (20%) to validation-jpg folder
def randomSelectForValidationSet(validation_fraction, current_wx):
    print('working on : ' + current_wx)
    current_train_dir = preprocess_train_dir + current_wx + '/'
    current_validation_dir = data_dir + 'preprocess/validation-jpg/'  + current_wx + '/'
    list_files = os.listdir(current_train_dir)
    random.shuffle(list_files) # In-place shuffle
    validation_size = int(round(len(list_files) * validation_fraction, 0))
    print('validation_size:' , validation_size)
    validation_files = list_files[0:validation_size]
    for curr_file in validation_files:
        src = current_train_dir + curr_file
        dest = current_validation_dir + curr_file
        #print ('src: ' + src)
        #print ('dest: ' + dest)
        move(src, dest)
    

In [86]:
wx_list = ['cloudy', 'partly_cloudy', 'clear', 'haze']
for current_wx in wx_list:
    randomSelectForValidationSet(0.20, current_wx)

working on : cloudy
('validation_size: %d', 461)
working on : partly_cloudy
('validation_size: %d', 896)
working on : clear
('validation_size: %d', 3599)
working on : haze
('validation_size: %d', 378)


In [87]:
print('# File sizes')
for f in os.listdir(preprocess_train_dir):
    if not os.path.isdir(preprocess_train_dir + f):
        print(f.ljust(30) + str(round(os.path.getsize(preprocess_train_dir + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize(preprocess_train_dir+f+'/'+x)/1000000 for x in os.listdir(preprocess_train_dir + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))

# File sizes
clear                         0.0MB (14396 files)
cloudy                        0.0MB (1845 files)
haze                          0.0MB (1513 files)
partly_cloudy                 0.0MB (3584 files)
